In [1]:
import numpy as np
import pandas as pd
import warnings
import datetime
import time
warnings.simplefilter('ignore')
import math
from statistics import mean
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
from sklearn.metrics import mean_squared_error


#from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
import category_encoders as ce

from sklearn.ensemble import RandomForestRegressor

In [2]:
status = pd.read_csv('../../../status.csv')

#statusのyear, month, dayを結合してdatetime型に
status['date'] = status['year'].astype(str) + '/' + status['month'].astype(str).str.zfill(2).astype(str) + '/' + status['day'].astype(str).str.zfill(2).astype(str) + '/' + status['hour'].astype(str).str.zfill(2).astype(str)
status['date'] = pd.to_datetime(status['date'], format='%Y/%m/%d/%H', infer_datetime_format=True)

#曜日を追加するための関数を定義
def get_weekday_jp(dt):
    w_list = ['月曜日', '火曜日', '水曜日', '木曜日', '金曜日', '土曜日', '日曜日']
    return(w_list[dt.weekday()])

#dateから曜日情報を取得
status["weekday"] = status["date"].apply(get_weekday_jp)

main_df = status[['date','hour', 'station_id', 'bikes_available', 'weekday', 'day']]

#カテゴリ変数をダミー変数化
main_df = pd.get_dummies(main_df)

stationid_set = [0, 15, 32, 36, 38, 40, 43, 49, 54, 56] 
dt_now = datetime.datetime.now()

print(dt_now)

2022-01-31 00:48:22.623507


In [3]:
for station_id in stationid_set:
    train_X = main_df[main_df['station_id'] == station_id]
    train_lag = train_X[train_X['date'] < '2013-10-01']['bikes_available']

    train_learn = train_X[train_X['date'] < '2013-10-01']
    train_learn = train_learn[train_learn['date'] >= '2013-09-02']
    
    i = 1
    while i < 25:
        row_name = "lag_" + str(i) + "hour"
        set = train_lag[24-i:720-i]
        train_learn[row_name] = set.values
        i += 1
    i = 0    
    
    train_learn_y = train_learn["bikes_available"]
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.drop("date",axis=1)
    train_learn_x = train_learn_x.drop("bikes_available",axis=1)    

    regr = RandomForestRegressor(max_depth=15, random_state=0)
    regr.fit(train_learn_x, train_learn_y)
    predictions = regr.predict(train_learn_x)
    train_learn['predict_24'] = predictions
    train_file_name = "Landom_24_predict/Landom_train_stationid" + str(station_id) + "_201309.csv" 
    train_learn.to_csv(train_file_name)
    
    test_lag = main_df[main_df['station_id'] == station_id][main_df['date'] < '2013-10-01']
    test_lag = test_lag[test_lag['date'] >= '2013-09-30']['bikes_available']     
    test = main_df[main_df['station_id'] == station_id][main_df['date'] < '2013-10-08']
    test = test[test['date'] >= '2013-10-01']
    test_y = test["bikes_available"]
    test_x = test.drop("station_id",axis=1)
    test_x = test_x.drop("date",axis=1)
    test_x = test_x.drop("bikes_available",axis=1)    

    pred_lightgbm_test = []
    len(pred_lightgbm_test)
    i=0
    while i < 168:
        test_x_line = test_x[i:i+1]

        lag = []
        for j in pred_lightgbm_test:
            lag.insert(0, j)

        for k in range(24 - len(pred_lightgbm_test)):
            lag.append(test_lag[23-k:24-k].values)
        s = 1
        while s < 25:
            row_name = "lag_" + str(s) + "hour"
            test_x_line[row_name] = lag[s-1]
            s += 1

        pred_lightgbm_test_lag_line = regr.predict(test_x_line)
        pred_lightgbm_test =np.append(pred_lightgbm_test, pred_lightgbm_test_lag_line)
        i += 1

    test['predict_24'] = pred_lightgbm_test
    test_file_name = "Landom_24_predict/Landom_test_stationid" + str(station_id) + "_201309.csv" 
    test.to_csv(test_file_name)
    print(dt_now)

2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902


In [4]:
for station_id in stationid_set:
    train_X = main_df[main_df['station_id'] == station_id]
    train_lag = train_X[train_X['date'] < '2014-02-01']
    train_lag = train_lag[train_lag['date'] >= '2014-01-01']['bikes_available']
    train_learn = train_X[train_X['date'] < '2014-02-01']
    train_learn = train_learn[train_learn['date'] >= '2014-01-02']
    
    i = 1
    while i < 25:
        row_name = "lag_" + str(i) + "hour"
        set = train_lag[24-i:744-i]
        train_learn[row_name] = set.values
        i += 1
    i = 0    
    
    train_learn_y = train_learn["bikes_available"]
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.drop("date",axis=1)
    train_learn_x = train_learn_x.drop("bikes_available",axis=1)    

    regr = RandomForestRegressor(max_depth=15, random_state=0)
    regr.fit(train_learn_x, train_learn_y)
    predictions = regr.predict(train_learn_x)
    train_file_name = "Landom_24_predict/Landom_train_stationid" + str(station_id) + "_201401.csv" 
    train_learn.to_csv(train_file_name)
    
    test_lag = main_df[main_df['station_id'] == station_id][main_df['date'] < '2014-02-01']
    test_lag = test_lag[test_lag['date'] >= '2014-01-31']['bikes_available']     
    test = main_df[main_df['station_id'] == station_id][main_df['date'] < '2014-02-08']
    test = test[test['date'] >= '2014-02-01']
    test_y = test["bikes_available"]
    test_x = test.drop("station_id",axis=1)
    test_x = test_x.drop("date",axis=1)
    test_x = test_x.drop("bikes_available",axis=1)    

    pred_lightgbm_test = []
    len(pred_lightgbm_test)
    i=0
    while i < 168:
        test_x_line = test_x[i:i+1]

        lag = []
        for j in pred_lightgbm_test:
            lag.insert(0, j)

        for k in range(24 - len(pred_lightgbm_test)):
            lag.append(test_lag[23-k:24-k].values)
        s = 1
        while s < 25:
            row_name = "lag_" + str(s) + "hour"
            test_x_line[row_name] = lag[s-1]
            s += 1

        pred_lightgbm_test_lag_line = regr.predict(test_x_line)
        pred_lightgbm_test =np.append(pred_lightgbm_test, pred_lightgbm_test_lag_line)
        i += 1

    test['predict_24'] = pred_lightgbm_test
    test_file_name = "Landom_24_predict/Landom_test_stationid" + str(station_id) + "_201401.csv" 
    test.to_csv(test_file_name)
    print(dt_now)

2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902


In [5]:
for station_id in stationid_set:
    train_X = main_df[main_df['station_id'] == station_id]
    train_lag = train_X[train_X['date'] < '2014-06-01']
    train_lag = train_lag[train_lag['date'] >= '2014-05-01']['bikes_available']
    train_learn = train_X[train_X['date'] < '2014-06-01']
    train_learn = train_learn[train_learn['date'] >= '2014-05-02']
    
    i = 1
    while i < 25:
        row_name = "lag_" + str(i) + "hour"
        set = train_lag[24-i:744-i]
        train_learn[row_name] = set.values
        i += 1
    i = 0    
    
    train_learn_y = train_learn["bikes_available"]
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.drop("date",axis=1)
    train_learn_x = train_learn_x.drop("bikes_available",axis=1)    

    regr = RandomForestRegressor(max_depth=15, random_state=0)
    regr.fit(train_learn_x, train_learn_y)
    predictions = regr.predict(train_learn_x)
    train_learn['predict_24'] = predictions
    train_file_name = "Landom_24_predict/Landom_train_stationid" + str(station_id) + "_201405.csv" 
    train_learn.to_csv(train_file_name)
    
    test_lag = main_df[main_df['station_id'] == station_id][main_df['date'] < '2014-06-01']
    test_lag = test_lag[test_lag['date'] >= '2014-05-31']['bikes_available']     
    test = main_df[main_df['station_id'] == station_id][main_df['date'] < '2014-06-08']
    test = test[test['date'] >= '2014-06-01']
    test_y = test["bikes_available"]
    test_x = test.drop("station_id",axis=1)
    test_x = test_x.drop("date",axis=1)
    test_x = test_x.drop("bikes_available",axis=1)    

    pred_lightgbm_test = []
    len(pred_lightgbm_test)
    i=0
    while i < 168:
        test_x_line = test_x[i:i+1]

        lag = []
        for j in pred_lightgbm_test:
            lag.insert(0, j)

        for k in range(24 - len(pred_lightgbm_test)):
            lag.append(test_lag[23-k:24-k].values)
        s = 1
        while s < 25:
            row_name = "lag_" + str(s) + "hour"
            test_x_line[row_name] = lag[s-1]
            s += 1

        pred_lightgbm_test_lag_line = regr.predict(test_x_line)
        pred_lightgbm_test =np.append(pred_lightgbm_test, pred_lightgbm_test_lag_line)
        i += 1

    test['predict_24'] = pred_lightgbm_test
    test_file_name = "Landom_24_predict/Landom_test_stationid" + str(station_id) + "_201405.csv" 
    test.to_csv(test_file_name)
    print(dt_now)

2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902
2022-01-11 07:07:21.929902


In [4]:
for station_id in stationid_set:
    train_X = main_df[main_df['station_id'] == station_id]
    train_learn = train_X[train_X['date'] < '2013-10-01']
    train_learn = train_learn[train_learn['date'] >= '2013-09-02']
    train_learn_y = train_learn["bikes_available"]
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.drop("date",axis=1)
    train_learn_x = train_learn_x.drop("bikes_available",axis=1)    

    regr = RandomForestRegressor(max_depth=15, random_state=0)
    regr.fit(train_learn_x, train_learn_y)
    predictions = regr.predict(train_learn_x)
    train_learn['predict_24'] = predictions
    train_file_name = "Landom_24_predict/nomal/Landom_train_stationid" + str(station_id) + "_201309.csv" 
    train_learn.to_csv(train_file_name)
    
    test = main_df[main_df['station_id'] == station_id][main_df['date'] < '2013-10-08']
    test = test[test['date'] >= '2013-10-01']
    test_y = test["bikes_available"]
    test_x = test.drop("station_id",axis=1)
    test_x = test_x.drop("date",axis=1)
    test_x = test_x.drop("bikes_available",axis=1)    
    pred_lightgbm_test = regr.predict(test_x)

    test['predict_24'] = pred_lightgbm_test
    test_file_name = "Landom_24_predict/nomal/Landom_test_stationid" + str(station_id) + "_201309.csv" 
    test.to_csv(test_file_name)

In [5]:
for station_id in stationid_set:
    train_X = main_df[main_df['station_id'] == station_id]
    train_learn = train_X[train_X['date'] < '2014-02-01']
    train_learn = train_learn[train_learn['date'] >= '2014-01-02']
    train_learn_y = train_learn["bikes_available"]
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.drop("date",axis=1)
    train_learn_x = train_learn_x.drop("bikes_available",axis=1)    

    regr = RandomForestRegressor(max_depth=15, random_state=0)
    regr.fit(train_learn_x, train_learn_y)
    predictions = regr.predict(train_learn_x)
    train_learn['predict_24'] = predictions
    train_file_name = "Landom_24_predict/nomal/Landom_train_stationid" + str(station_id) + "_201401.csv" 
    train_learn.to_csv(train_file_name)
    
    test = main_df[main_df['station_id'] == station_id][main_df['date'] < '2014-02-08']
    test = test[test['date'] >= '2014-02-01']
    test_y = test["bikes_available"]
    test_x = test.drop("station_id",axis=1)
    test_x = test_x.drop("date",axis=1)
    test_x = test_x.drop("bikes_available",axis=1)    
    pred_lightgbm_test = regr.predict(test_x)

    test['predict_24'] = pred_lightgbm_test
    test_file_name = "Landom_24_predict/nomal/Landom_test_stationid" + str(station_id) + "_201401.csv" 
    test.to_csv(test_file_name)

In [9]:
start = time.time()
for station_id in stationid_set:
    train_X = main_df[main_df['station_id'] == station_id]
    train_learn = train_X[train_X['date'] < '2014-06-01']
    train_learn = train_learn[train_learn['date'] >= '2014-05-02']
    train_learn_y = train_learn["bikes_available"]
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.drop("date",axis=1)
    train_learn_x = train_learn_x.drop("bikes_available",axis=1)    

    regr = RandomForestRegressor(max_depth=15, random_state=0)
    regr.fit(train_learn_x, train_learn_y)
    predictions = regr.predict(train_learn_x)
    train_learn['predict_24'] = predictions
    train_file_name = "Landom_24_predict/nomal/Landom_train_stationid" + str(station_id) + "_201405.csv" 
    train_learn.to_csv(train_file_name)
    
    test = main_df[main_df['station_id'] == station_id][main_df['date'] < '2014-06-08']
    test = test[test['date'] >= '2014-06-01']
    test_y = test["bikes_available"]
    test_x = test.drop("station_id",axis=1)
    test_x = test_x.drop("date",axis=1)
    test_x = test_x.drop("bikes_available",axis=1)    
    pred_lightgbm_test = regr.predict(test_x)

    test['predict_24'] = pred_lightgbm_test
    test_file_name = "Landom_24_predict/nomal/Landom_test_stationid" + str(station_id) + "_201405.csv" 
    test.to_csv(test_file_name)
elapsed_time = time.time() - start
print(elapsed_time)

1.7183868885040283


In [3]:
start = time.time()
for station_id in stationid_set:
    train_X = main_df[main_df['station_id'] == station_id]
    train_learn = train_X[train_X['date'] < '2013-10-01']
    train_learn = train_learn[train_learn['date'] >= '2013-09-02']
    train_learn_y = train_learn["bikes_available"]
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.drop("date",axis=1)
    train_learn_x = train_learn_x.drop("bikes_available",axis=1)    

    regr = RandomForestRegressor(max_depth=15, random_state=0)
    regr.fit(train_learn_x, train_learn_y)
    predictions = regr.predict(train_learn_x)
    train_learn['predict_24'] = predictions
    train_file_name = "Landom_24_predict/nomal/Landom_train_stationid" + str(station_id) + "_201309.csv" 
#    train_learn.to_csv(train_file_name)
    
    test = main_df[main_df['station_id'] == station_id][main_df['date'] < '2013-10-08']
    test = test[test['date'] >= '2013-10-01']
    test_y = test["bikes_available"]
    test_x = test.drop("station_id",axis=1)
    test_x = test_x.drop("date",axis=1)
    test_x = test_x.drop("bikes_available",axis=1)    
    pred_lightgbm_test = regr.predict(test_x)

    test['predict_24'] = pred_lightgbm_test
    test_file_name = "Landom_24_predict/nomal/Landom_test_stationid" + str(station_id) + "_201309.csv" 
#    test.to_csv(test_file_name)
elapsed_time = time.time() - start
print(elapsed_time)

1.6183626651763916


In [5]:
#計測用2013年10月
start = time.time()
for station_id in stationid_set:
    train_X = main_df[main_df['station_id'] == station_id]
    train_lag = train_X[train_X['date'] < '2013-10-01']['bikes_available']

    train_learn = train_X[train_X['date'] < '2013-10-01']
    train_learn = train_learn[train_learn['date'] >= '2013-09-02']
    
    i = 1
    while i < 25:
        row_name = "lag_" + str(i) + "hour"
        set = train_lag[24-i:720-i]
        train_learn[row_name] = set.values
        i += 1
    i = 0    
    
    train_learn_y = train_learn["bikes_available"]
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.drop("date",axis=1)
    train_learn_x = train_learn_x.drop("bikes_available",axis=1)    

    regr = RandomForestRegressor(max_depth=15, random_state=0)
    regr.fit(train_learn_x, train_learn_y)
    predictions = regr.predict(train_learn_x)
    train_learn['predict_24'] = predictions
    train_file_name = "Landom_24_predict/Landom_train_stationid" + str(station_id) + "_201309.csv" 
#    train_learn.to_csv(train_file_name)
    
    test_lag = main_df[main_df['station_id'] == station_id][main_df['date'] < '2013-10-01']
    test_lag = test_lag[test_lag['date'] >= '2013-09-30']['bikes_available']     
    test = main_df[main_df['station_id'] == station_id][main_df['date'] < '2013-10-08']
    test = test[test['date'] >= '2013-10-01']
    test_y = test["bikes_available"]
    test_x = test.drop("station_id",axis=1)
    test_x = test_x.drop("date",axis=1)
    test_x = test_x.drop("bikes_available",axis=1)    

    pred_lightgbm_test = []
    len(pred_lightgbm_test)
    i=0
    while i < 168:
        test_x_line = test_x[i:i+1]

        lag = []
        for j in pred_lightgbm_test:
            lag.insert(0, j)

        for k in range(24 - len(pred_lightgbm_test)):
            lag.append(test_lag[23-k:24-k].values)
        s = 1
        while s < 25:
            row_name = "lag_" + str(s) + "hour"
            test_x_line[row_name] = lag[s-1]
            s += 1

        pred_lightgbm_test_lag_line = regr.predict(test_x_line)
        pred_lightgbm_test =np.append(pred_lightgbm_test, pred_lightgbm_test_lag_line)
        i += 1

    test['predict_24'] = pred_lightgbm_test
    test_file_name = "Landom_24_predict/Landom_test_stationid" + str(station_id) + "_201309.csv" 
#    test.to_csv(test_file_name)
elapsed_time = time.time() - start
print(elapsed_time)

24.19123601913452
